In [2]:
from google.colab import drive
import os
drive.flush_and_unmount()
drive.mount('/content/drive/')

aria_path = "/content/drive/My Drive/Aria Workshop"

# + "streaming1.json"


Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive/


In [4]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00


In [5]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [6]:
import os
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate

os.environ["OPENAI_API_KEY"] = "sk-"
chat_model = ChatOpenAI()

In [7]:
"""Example usage of GPT4-V API.

Usage:

    OPENAI_API_KEY=<your_api_key> python3 gpt4v.py \
        [<path/to/image1.png>] [<path/to/image2.jpg>] [...] "text prompt"

Example:

    OPENAI_API_KEY=xxx python3 gpt4v.py photo.png "What's in this photo?"
"""

from pprint import pprint
import base64
import json
import mimetypes
import os
import requests
import sys


api_key = os.getenv("OPENAI_API_KEY")


def encode_image(image_path: str):
    """Encodes an image to base64 and determines the correct MIME type."""
    mime_type, _ = mimetypes.guess_type(image_path)
    if mime_type is None:
        raise ValueError(f"Cannot determine MIME type for {image_path}")

    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
        return f"data:{mime_type};base64,{encoded_string}"


def create_payload(images: list[str], prompt: str, model="gpt-4-vision-preview", max_tokens=100, detail="high"):
    """Creates the payload for the API request."""
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
            ],
        },
    ]

    for image in images:
        base64_image = encode_image(image)
        messages[0]["content"].append({
            "type": "image_url",
            "image_url": {
                "url": base64_image,
                "detail": detail,
            }
        })

    return {
        "model": model,
        "messages": messages,
        "max_tokens": max_tokens
    }


def query_openai(payload):
    """Sends a request to the OpenAI API and prints the response."""
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response.json()





In [8]:
# compression of images are important step
prompt = "Your are an expert of meta campus. You will be given an image taken on the meta campus. Describe the object, the uniqueness of it. Don't say anything if you don't see, don't excuse youself. Format your answer as a JSON object like this {\"object\": \"sculpture of emoji\" , \"uniqueness\": \"a yellow huggingface emoji with heart\"}. If you don't know an element, just write None in place. Do not output anything else, just the pure JSON string starting with {"
image_paths = [aria_path+"/person_b/images/IMG_7931.JPEG"]
payload = create_payload(image_paths, prompt)
response = query_openai(payload)
print(response)

{'id': 'chatcmpl-8TYSC4LKwjKXKBngFa0koJppKRPq8', 'object': 'chat.completion', 'created': 1702054644, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 881, 'completion_tokens': 26, 'total_tokens': 907}, 'choices': [{'message': {'role': 'assistant', 'content': '{"object": "sculpture", "uniqueness": "wooden spiral resembling a large spinning top or drill bit"}'}, 'finish_details': {'type': 'stop', 'stop': '<|fim_suffix|>'}, 'index': 0}]}


In [13]:
import cv2
import glob
import json
from tqdm import tqdm



path =aria_path+"/person_a/images/*.jpg"

user1_art = []

for file in tqdm(glob.glob(path)):
  payload = create_payload([file], prompt)
  response = query_openai(payload)
  try:
    artwork= response["choices"][0]["message"]["content"]
    user1_art.append(json.loads(artwork))
  except KeyboardInterrupt:
    break
  except:
      continue
   #print(file)
   #a= cv2.imread(file)


100%|██████████| 10/10 [01:18<00:00,  7.84s/it]


In [14]:
user1_art

[{'object': 'arcade machines',
  'uniqueness': "classic-style stand-up cabinets, one showing 'Street Fighter' game"},
 {'object': 'sculpture of emoji',
  'uniqueness': 'a large emoji with a laughing expression and tears, displayed on the side of a building'},
 {'object': "sculpture of a 'like' icon",
  'uniqueness': "a large, red three-dimensional 'thumbs up' with a white heart in the center"},
 {'object': 'sculpture of notification icon',
  'uniqueness': "a large, three-dimensional depiction of a notification or 'like' icon, typically seen on social media platforms, with a heart shape in the center"},
 {'object': 'sculpture',
  'uniqueness': 'a large, three-dimensional heart shape in a red frame'},
 {'object': 'sculpture with like symbol',
  'uniqueness': 'a large three-dimensional version of the like icon, with a red background and white heart in the center'},
 {'object': 'bicycles',
  'uniqueness': 'a row of parked bicycles with various colors, featuring upright handlebars and front

In [10]:
artwork

'{"object": "sculpture of an emoji", "uniqueness": "a yellow emoji with heart-shaped eyes and an open-mouthed smile"}'

In [15]:
with open("person_a_meta.json", "w") as f:
  json.dump(user1_art, f)

In [ ]:
user1_art

[{'cuisine': 'Japanese',
  'Dish': 'Steamed Rice',
  'Ingredients': ['rice', 'water'],
  'Cooking': 'Steaming'},
 {'cuisine': 'None', 'Dish': 'None', 'Ingredients': 'None', 'Cooking': 'None'},
 {'cuisine': 'Italian',
  'Dish': 'Spaghetti Carbonara',
  'Ingredients': ['spaghetti',
   'eggs',
   'pecorino cheese',
   'guanciale',
   'black pepper',
   'peas'],
  'Cooking': 'Boiled and sautéed'},
 {'cuisine': 'Italian',
  'Dish': 'Spaghetti Carbonara',
  'Ingredients': ['spaghetti',
   'guanciale or pancetta',
   'eggs',
   'Pecorino Romano cheese',
   'black pepper'],
  'Cooking': 'Boiled and tossed'},
 {'cuisine': 'Italian',
  'Dish': 'Spaghetti Carbonara',
  'Ingredients': ['spaghetti',
   'guanciale or pancetta',
   'eggs',
   'pecorino romano cheese',
   'black pepper'],
  'Cooking': 'Boiled and sauteed'}]

In [ ]:
import pandas as pd
df = pd.read_json(objects_dict)
df.to_csv(objects_save_path)
display(df)

,timestamps,objects
0,1.641293e+12,"[book, handle, chopping_board, refrigerator, p..."
1,1.643188e+12,"[tomato, carton, jar, refrigerator, bowl, bott..."
2,1.645082e+12,"[strawberry, tomato, person, remote_control, d..."
3,1.646977e+12,"[strawberry, tomato, remote_control, jar, refr..."
4,1.648872e+12,"[lettuce, tomato, vent, egg, salad, drawer, re..."
5,1.650766e+12,"[tomato, person, vent, watch, refrigerator, po..."
6,1.652661e+12,"[tomato, person, remote_control, egg, refriger..."
7,1.654556e+12,"[lettuce, tomato, person, tag, remote_control,..."
8,1.656451e+12,"[lettuce, strawberry, tomato, person, vent, ta..."
9,1.658345e+12,"[tomato, person, vent, hinge, jar, toilet_tiss..."


### We will aggregate these object list, and feed to an LLM.

In [ ]:
agg_obj_list = list(set([item for sublist in objects_dict['objects'] for item in sublist]))

print(agg_obj_list)

['lemon', 'carrot', 'handle', 'bracelet', 'drawer', 'yogurt', 'cover', 'fishbowl', 'tinfoil', 'flowerpot', 'egg', 'tag', 'jar', 'toilet_tissue', 'apple', 'refrigerator', 'plastic_bag', 'ring', 'bucket', 'bottle_cap', 'orange_(fruit)', 'persimmon', 'book', 'hinge', 'cup', 'remote_control', 'chopping_board', 'bolt', 'aerosol_can', 'magnet', 'potato', 'can', 'box', 'bowl', 'strawberry', 'lettuce', 'tomato', 'vent', 'carton', 'salad', 'matchbox', 'watch', 'onion', 'bottle', 'person', 'toy']


## Step 9. Install Langchain

## Step 10. Install OpenAI to use with Langchain

## Step 11. Get ingredients and recipes

We ask LLM to get food ingredients from the object detection result (`agg_obj_list`), and then ask it to provide recipes.


In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate

os.environ["OPENAI_API_KEY"] = "sk-"
chat_model = ChatOpenAI()

agg_obj_list= ["lettuce", "dog"]

response = chat_model.predict(f"List food ingredients in this list: {', '.join(agg_obj_list)}")
print(response)

lettuce


In [ ]:
"""Example usage of GPT4-V API.

Usage:

    OPENAI_API_KEY=<your_api_key> python3 gpt4v.py \
        [<path/to/image1.png>] [<path/to/image2.jpg>] [...] "text prompt"

Example:

    OPENAI_API_KEY=xxx python3 gpt4v.py photo.png "What's in this photo?"
"""

from pprint import pprint
import base64
import json
import mimetypes
import os
import requests
import sys


api_key = os.getenv("OPENAI_API_KEY")


def encode_image(image_path: str):
    """Encodes an image to base64 and determines the correct MIME type."""
    mime_type, _ = mimetypes.guess_type(image_path)
    if mime_type is None:
        raise ValueError(f"Cannot determine MIME type for {image_path}")

    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
        return f"data:{mime_type};base64,{encoded_string}"


def create_payload(images: list[str], prompt: str, model="gpt-4-vision-preview", max_tokens=100, detail="high"):
    """Creates the payload for the API request."""
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
            ],
        },
    ]

    for image in images:
        base64_image = encode_image(image)
        messages[0]["content"].append({
            "type": "image_url",
            "image_url": {
                "url": base64_image,
                "detail": detail,
            }
        })

    return {
        "model": model,
        "messages": messages,
        "max_tokens": max_tokens
    }


def query_openai(payload):
    """Sends a request to the OpenAI API and prints the response."""
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response.json()





In [ ]:
# compression of images are important step
prompt = "are those two artworks similar? answer by yes or no + painting style in two words maximum"
image_paths = ["jaune-rouge-bleu.jpg", "jaune-rouge-bleu2.jpg"]
payload = create_payload(image_paths, prompt)
response = query_openai(payload)
print(response)

{'id': 'chatcmpl-8TEVQEoFN4nZ7T7vdE50IoIi9wwBi', 'object': 'chat.completion', 'created': 1701977964, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 535, 'completion_tokens': 4, 'total_tokens': 539}, 'choices': [{'message': {'role': 'assistant', 'content': 'Yes, Abstract.'}, 'finish_details': {'type': 'stop', 'stop': '<|fim_suffix|>'}, 'index': 0}]}


In [ ]:
template = "You are a helpful culinary assistant that provides recipes given a list of ingredients."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])


chain = chat_prompt | ChatOpenAI()
recipe = chain.invoke({"text": response})

In [ ]:
print(recipe.content)

Here are a few recipes you can make with the ingredients you provided:

1. Greek Potato Salad:
- Ingredients: potato, lettuce, onion, yogurt, lemon
- Instructions: Boil the potatoes until tender. Let them cool, then cube them. Mix together the potatoes, lettuce, finely chopped onion, and a dressing made with yogurt and lemon juice. Season with salt and pepper to taste. Serve chilled.

2. Carrot Apple Muffins:
- Ingredients: carrot, apple, egg, yogurt
- Instructions: Preheat the oven to 350°F (175°C). Grate the carrots and apples. In a mixing bowl, beat the eggs and add yogurt. Then, add the grated carrots and apples. Mix well. Pour the batter into muffin cups and bake for about 20 minutes or until a toothpick inserted comes out clean.

3. Persimmon Salad:
- Ingredients: persimmon, lettuce, tomato, onion
- Instructions: Slice the persimmons, tomatoes, and onions. Arrange them on a bed of lettuce. Drizzle with lemon juice or a light dressing of your choice. Serve as a refreshing salad.



## Step 12. Personalized recipes
You have the ability to personalize the recipe based on your individual preferences. In this example, we demonstrate how to input preferences manually, but in an ideal scenario, this information would be obtained from your digital data.

In [ ]:
template = """
You are a helpful culinary assistant that provides recipes given a list of ingredients.
I like healthy meals, and I enjoy mediterranean food.
"""
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])


chain = chat_prompt | ChatOpenAI()
recipe = chain.invoke({"text": response})
print(recipe.content)

Here's a Mediterranean-inspired recipe that incorporates some of the ingredients you provided: Lemon Potato Salad.

Ingredients:
- 4 potatoes, boiled and diced
- 1 small onion, finely chopped
- 1 lemon, juiced and zested
- 2 tablespoons extra virgin olive oil
- Salt and pepper, to taste
- 1 cup lettuce, chopped
- 1 small apple, diced
- 1 carrot, grated
- 1 persimmon, diced

Instructions:
1. In a large mixing bowl, combine the diced potatoes, chopped onion, lemon juice, lemon zest, olive oil, salt, and pepper. Mix well to coat the potatoes.
2. Add the lettuce, diced apple, grated carrot, and diced persimmon to the bowl. Toss gently to combine all the ingredients.
3. Adjust the seasoning if needed, and refrigerate for at least 30 minutes to allow the flavors to meld together.
4. Serve chilled as a side dish or as a light lunch option.

Optional: You can also add chopped tomatoes and sliced strawberries to add more color and flavor to the salad. Enjoy your healthy Mediterranean meal!
